# Audiobook Analysis

In [2]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

# Load the data
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')

unscaled_inputs_all = raw_csv_data[:,1:-1]

targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [3]:
# Count how many targets are 1 
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 
zero_targets_counter = 0


indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [6]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)


test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1802.0 3579 0.5034925956971221
208.0 447 0.465324384787472
227.0 448 0.5066964285714286


### Save the three datasets in *.npz

In [7]:
#save as npz
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Data

In [8]:

data = np.load('Audiobooks_data_train.npz')

train_inputs = data['inputs'].astype(float)
train_targets = data['targets'].astype(int)


data = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = data['inputs'].astype(float), data['targets'].astype(int)

data = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = data['inputs'].astype(float), data['targets'].astype(int)
print(train_inputs.shape)
print(validation_inputs.shape)
print(train_targets.shape)
print(validation_targets.shape)
print(train_inputs)

(3579, 10)
(447, 10)
(3579,)
(447,)
[[ 1.27894497  0.41646744 -0.39082475 ...  0.7102813  -0.20183481
   0.46453021]
 [-0.64419501 -0.67316726 -0.39082475 ... -0.41569922 -0.20183481
   1.21503662]
 [-0.85787724 -0.79423778  0.39886313 ...  3.10298991  2.20618899
   0.1136441 ]
 ...
 [ 1.27894497  0.41646744 -0.12924064 ... -0.41569922 -0.20183481
  -0.80255852]
 [-0.64419501 -0.67316726 -0.39082475 ... -0.41569922 -0.20183481
  -0.80255852]
 [-0.64419501 -0.67316726 -0.12924064 ... -0.41569922 -0.20183481
   2.52111269]]


### Model
Outline, optimizers, loss, early stopping and training

In [9]:
# Set the input and output sizes
input_size = 10
output_size = 2

hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([

                  tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                  tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
                  tf.keras.layers.Dense(output_size, activation='softmax')
                            ])



model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 2 
          )  

Epoch 1/100
36/36 - 0s - loss: 0.5635 - accuracy: 0.6770 - val_loss: 0.4892 - val_accuracy: 0.7562 - 426ms/epoch - 12ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4461 - accuracy: 0.7664 - val_loss: 0.4305 - val_accuracy: 0.7472 - 56ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4034 - accuracy: 0.7846 - val_loss: 0.3984 - val_accuracy: 0.8009 - 51ms/epoch - 1ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3821 - accuracy: 0.7963 - val_loss: 0.3777 - val_accuracy: 0.7942 - 53ms/epoch - 1ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3711 - accuracy: 0.7949 - val_loss: 0.3746 - val_accuracy: 0.7964 - 46ms/epoch - 1ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3611 - accuracy: 0.7999 - val_loss: 0.3642 - val_accuracy: 0.8009 - 48ms/epoch - 1ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3572 - accuracy: 0.8055 - val_loss: 0.3602 - val_accuracy: 0.7964 - 57ms/epoch - 2ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3523 - accuracy: 0.8086 - val_loss: 0.3553 - val_accuracy: 0.8121 - 49ms/epoch - 1ms/step
Epoch 9/100
36

In [10]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3210 - accuracy: 0.8393


In [11]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.32. Test accuracy: 83.93%
